> **在数据分析中，最有价值的是数据，并非分析方法。**

既然都已经知道了数据是最有价值的，那我们直接找到一些数据不就可以了么？

理想很丰满，现实很骨感。在真实的世界里，我们并不是就直接就有现成的并且格式规整的数据的，经常是需要我们自己去收集数据、整理数据。

我们这一个实战练习就是就是为了让大家学会如何去将多个原始的文件合并成我们最终需要进行分析的汇总文件，包括以下几点：

- 读取需要表格文件
- 合并多个表格文件
- 写入汇总表格文件

这里我们要对MovieLens数据集进行分析，简单介绍下这个数据集的由来，它由GroupLens研究组在明尼苏达大学组织收集的一些用户对一些电影的评分。这个数据集经常用在**推荐系统**的领域。

这里我对原始的数据集进行了处理，处理后一共是4个文件：links_1.csv, links_2.csv, movies.csv, ratings.csv。我们要做的就是将这些文件合并成一个文件，并将生成的文件保存，以便于我们接下来分析使用。

In [1]:
import pandas as pd
import numpy as np

## 读取原始文件

每个原始文件都是标准的csv文件，读取时直接调用 `pd.read_csv` 即可。

ratings.csv 是用户对电影评分的数据文件，包含四列，分别是 userId,movieId,rating,timestamp 。每一行数据表示用户在某个时间为某个电影打的分数。rating 属于 0.5-5.0 星之间，timestamp 表示 unix 时间戳（10位，精确到秒）。

这里将 ratings.csv 文件切分成了两部分：ratings_1.csv 和 ratings_2.csv。

注意：这两个文件中不存在重复的用户评分数据。

In [2]:
ratings_1 = pd.read_csv("../data/ml-latest-small/ratings_1.csv")
ratings_1.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [3]:
ratings_2 = pd.read_csv("../data/ml-latest-small/ratings_2.csv")
ratings_2.head(2)

,userId,movieId,rating,timestamp
0,367,911,NaN,1128633117
1,367,969,5.0,1128632828


movies.csv（电影文件）：电影文件包含三列，分别是 movieId,title,genres 。每一行数据表示这个电影的标题和体裁。一部电影的体裁（genres）可能有多个，多个之间使用 “|” 分隔，genres 可选的值为：Action, Adventure, Animation, Children's, Comedy, Crime, Documentary, Drama, Fantasy, Film-Noir, Horror, Musical, Mystery, Romance, Sci-Fi, Thriller, War, Western。

In [4]:
movies = pd.read_csv("../data/ml-latest-small/movies.csv")
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


links.csv（电影链接文件）：电影链接文件包含三列，分别是 movieId,imdbId,tmdbId 。每一行数据表示这个电影对应在 imdb 和 themoviedb 电影网站的 id。

In [5]:
links = pd.read_csv("../data/ml-latest-small/links.csv")
links.head(2)

,movieId,imdbId,tmdbId
0,1,114709.0,862.0
1,2,113497.0,8844.0


经过上面的操作后，我们得到了4个DataFrame：ratings_1、ratings_2、movies、links。

## 合并数据

当我们将这4个文件转为DataFrame之后，如何合并后生成一个汇总的DataFrame呢？

### 合并用户评分数据（拼接）

从上面可以看出来，ratings_1 和 ratings_2 都是用户对电影的评分数据，既然都是用户对电影的评分数据，并且字段相同，如何将它们合并成一个 DataFrame 呢？

先来看下 ratings_1 和 ratings_2 的数据条数吧!

In [6]:
print(len(ratings_1))
print(len(ratings_2))

50002
50002


可以看到，ratings_1 和 ratings_2 都是 50002 条数据，也就意味合并后会有 50002 + 50002 = 100004 （因为这两个文件不包含重复数据，所以直接相加即可）条用户对电影的评分记录。

调用 `append` 方法即可完成数据数据合并（拼接），同时我们可以指定参数 `ignore_index=True` 来重新生成索引。

In [7]:
ratings_info = ratings_1.append(ratings_2, ignore_index=True)
ratings_info.shape

(100004, 4)

In [8]:
ratings_info.tail()

,userId,movieId,rating,timestamp
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663
100003,671,6565,3.5,1074784724


合并完成后，总体条数是正确的。合并了用户对电影的评分数据后，我们来看下如何合并电影信息吧

### 合并电影信息（关联）

movies 和 links 这两个DataFrame都是关于电影的信息，只是一个是关于电影的标题和体裁，另一个是关于电影在 imdb 和 themoviedb 电影网站的 id。我们合并后，期望一部电影的标题、体裁以及在 imdb 和 themoviedb 电影网站的 id能够出现在一条数据中。如何实现呢？

调用 `merge` 方法即可完成合并（关联），不过在合并（关联）的时候需要考虑两点：

- 依据哪个字段合并
- 采用哪种方式合并（内连接[inner]、外连接[outer]、左连接[left]、右连接[right]）

我们这里是为了将电影的相关信息关联起来，所以依据 movieId 字段来关联。

那采用哪种方式来合并呢？我们来看下待合并的两个DataFrame中包含的不重复的 movieId 之间的关系。

In [9]:
# movies 包含的不重复的 movieId 个数
print(len(movies.movieId.unique()))
# links 包含的不重复的 movieId 个数
print(len(links.movieId.unique()))
# movies 和 links 交集中包含的不重复的 movieId 个数
print(len(np.intersect1d(movies.movieId.unique(), links.movieId.unique())))

9125
9125
9125


上面的统计结果显示 movies 和 links 关于 movieId 交集的个数与它们本身的去重后 movieId 个数相同，所以我们无论采用哪种合并方式，得到的结果都是一样的。所以我们设置参数 `how="inner"`表示采用内连接，设置参数 `on="movieId"` 表示依据 movieId 这个字段来关联。

In [10]:
movies_info = movies.merge(links, how="inner", on="movieId")
movies_info.shape

(9125, 5)

In [11]:
movies_info.head()

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709.0,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497.0,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228.0,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885.0,31357.0
4,5,Father of the Bride Part II (1995),Comedy,113041.0,11862.0


### 合并用户和电影信息（关联）

我们已经得到了用户对电影的评分数据以及电影的信息这两个 Dataframe，如何将这两个 DataFrame 合并在一起呢？

其实合并方法与上面合并电影信息类似，只需要确定关联的字段以及关联的方式即可。很明显，关联字段是 movieId，我们来确定下关联的方式。

In [12]:
# ratings_info 包含的不重复的 movieId 个数
print(len(ratings_info.movieId.unique()))
# movies_info 包含的不重复的 movieId 个数
print(len(movies_info.movieId.unique()))
# ratings_info 和 movies_info 交集中包含的不重复的 movieId 个数
print(len(np.intersect1d(ratings_info.movieId.unique(), movies_info.movieId.unique())))

9066
9125
9066


上面结果说明 ratings_info 包含的 movieId 是 movies_info 包含的 movieId 的真子集，也就是说有部分电影没有经过用户评分，这里我们想要保留有用户评分的电影的相关记录，没有用户评分的电影记录舍弃。

假定我们在合并时，假定 ratings_info 放在左边，movies_info 放在右边，这时候关联方式选择左连接[left]和内连接[inner]都可以满足上面的要求。

In [13]:
data = ratings_info.merge(movies_info, how="left", on="movieId")
data.shape

(100004, 8)

In [14]:
data.head()

,userId,movieId,rating,timestamp,title,genres,imdbId,tmdbId
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,112792.0,9909.0
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,33563.0,11360.0
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,117665.0,819.0
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,82340.0,1103.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,95765.0,11216.0


## 保存汇总数据

我们已经得到了合并后的所有数据，这时候我们需要将数据保存起来，以便于我们之后来分析使用。

这里我们将它存为一个标准的 csv 文件，直接调用 `to_csv` 方法即可，为了避免索引也被保存，所以指定参数 `index=False`。

In [15]:
data.to_csv("../data/ml-latest-small/user_for_movie_ratings_summary.csv", index=False)

![](image/公众号—AI派.jpg)